This file will be used for interacting with the SPCA class; the idea is to formalize all the computations and methods that we need in the SPCA class, and use this Juypiter Notebook for interacting with it. Think importing and transforming data, plotting, running the algorithms, etc.

In [1]:
# Import SPCA class and other relevant packages
from spca import Spca
from elasticnet import ElasticNet
import pandas as pd
import numpy as np
import plotnine as pn

ModuleNotFoundError: No module named 'numpy'

In [21]:
# Import both the pitprops and gene datasets; for the pitprops dataset, use the first column as the indices.
pitprops = pd.read_csv("data/pitprops.csv", index_col=0)
genedata = pd.read_csv("data/GCM_Total.csv")

To-do w.r.t. application of the SPCA algorithm
- Make some descriptive plots/tables for both datasets; this might be a bit less trivial for the pitprops dataset since it is just a correlation matrix, not a real dataset. We could also try to find the original pitprops dataset, this would make writing the SPCA class a bit easier as well since we now need to do different computations for the two datasets.
- Run the SPCA class on both datasets to obtain the results from Zou, Hastie and Tibshirani (2006)
- For comparison, do the same using a built-in package; SPCA should be available in scikit-learn (See https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.SparsePCA.html)
    - Compare results between hand-written and built-in
    - Compare runtime between hand-written and built-in (Should be trivial, Jupyter should provide you with the runtime of a block of code)
- ..

In [ ]:
### Descriptive statistics and plots

pitprops.describe
genedata.describe


Comments and analysis of descriptives and plots

In [ ]:
### Run the main algorithm

# For both datasets separately:

# Calculate SVD for initial loadings
Ploadings = np.linalg.svd(pitprops)
Gloadings = np.linalg.svd(genedata)

# Run SPCA algorithm
Presult = Spca.SPCAalgo(Ploadings)
Gresult = Spca.SPCAalgo(Gloadings)

# Report results

print(Presult)
print(Gresult)

Comments and analysis of algorithm and runtime

In [ ]:
# After those 5 steps, we also want to compute some measures like number of nonzero loadings, variance and cumulative variance. We also need to think how we want
# to visualize the SPCA outcomes for the gene dataset, since we can't simply show ~16,000 loadings

# Calculate performance measures
# Maybe elbow plots?

Comments and analysis of results and performance measures

Possible extensions:

1. Compare runtime between packages & our implementation
2. Compare results between paper & our implementation
3. Vary k, lambda, stopping criterion
4. Calculate SVD without packages
